Лабораторная работа #4
Цель: Реализовать базовый механизм поиска по датасету с помощью классических методов информационного поиска (ключевые слова, TF-IDF, BM25). Оценить качество поиска с помощью простых метрик.

Задачи:

Индексирование коллекции: На основе очищенных данных из ЛР3 сформировать индекс для быстрого поиска по тексту вопросов. Реализовать инвертированный индекс. Каждому вопросу сопоставить список значимых слов или термов с их весами (частота, TF-IDF и т.д.).

Реализация поиска: Разработать функцию, которая по строке запроса (вопрос на естественном языке) находит наиболее похожие вопросы из базы:

Преобразовать запрос так же, как документы (очистка, токенизация, вычисление TF-IDF-вектора).

Вычислить сходство запроса с каждым вопросом (например, косинусное сходство между TF-IDF векторами или BM25 скоринг).

Выдать топ-5 или топ-10 наиболее релевантных вопросов из датасета, отсортированных по убыванию схожести.

Извлечение ответа: Используя результаты поиска, получить связанные ответы. Например, для каждого найденного похожего вопроса извлечь его самый рейтинговый или принятый ответ из датасета. Таким образом, по пользовательскому запросу система вернет список потенциально полезных Q&A пар.

Оценка качества (метрики): Провести базовую оценку работы поискового механизма. Можно составить небольшой набор тестовых запросов и вручную проверить, насколько выданные ответы релевантны. Если в датасете есть пометка дубликатов вопросов или тегов, использовать их для проверки (например, запросить вопрос и проверить, находится ли его дубликат в топ-результатах). Рассмотреть метрики вроде Precision@5 (точность на топ-5 результатов) или MRR – на небольшом наборе примеров.

Документация результатов: Описать реализованный поиск и приведенные примеры работы. Сделать выводы о плюсах и минусах классического подхода: быстрое выполнение и простота, но возможные проблемы (неучет синонимов, контекста и т.д.). Подготовить код и отчет, которые станут основой для сравнения с векторным подходом в следующих лабораторных.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Лабаораторная работа #4

## Загрузка данных.

In [2]:
# Датасет вопросов
questions = pd.read_csv('../lab3/dataset/Questions.csv', encoding='latin1')
# Выводим первые 5 строк
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [3]:
# Очищенный датасет вопросов
questions_clear = pd.read_csv('../lab3/dataset/Questions_clear.csv', encoding='latin1')
# Выводим первые 5 строк
questions_clear.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,write database generation script sql want exec...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,really good tutorial explain branching merge a...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,anyone get experience create sqlbased aspnet s...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,something pseudosolved many time never quite f...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,little game write c us database backend tradin...


In [4]:
# Датасет ответов
answers = pd.read_csv('../lab3/dataset/Answers.csv', encoding='latin1')
# Выводим первые 5 строк
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [82]:
# Очищенный датасет ответов
answers_clear = pd.read_csv('../lab3/dataset/Answers_clear.csv', encoding='latin1')
# Выводим первые 5 строк
answers_clear.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,version control subversion good resource sourc...
1,124,26.0,2008-08-01T16:09:47Z,80,12,wound use kind hack actually work pretty well ...
2,199,50.0,2008-08-01T19:36:46Z,180,1,read somewhere human eye distinguish less 4 va...
3,269,91.0,2008-08-01T23:49:57Z,260,4,yes thought soon figure another domainspecific...
4,307,49.0,2008-08-02T01:49:46Z,260,28,oleg shilos c script solution code project rea...


In [9]:
# Датасет тегов
tags = pd.read_csv('../lab3/dataset/Tags.csv', encoding='latin1')
# Выводим первые 5 строк
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


Функция для очистки текста.

In [8]:
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')  # Загрузка стоп-слов
nltk.download('wordnet')  # Загрузка WordNet
nltk.download('averaged_perceptron_tagger_eng')  # Загрузка тегов

[nltk_data] Downloading package stopwords to C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [11]:
stop_words = set(stopwords.words('english'))  # Загрузка стоп-слов
contractions = ['ive', 'youre', 'theyre', 'hes', 'shes', 'its', 'weve', 'theyve', 'im', 'isnt', 'wasnt', 'werent',
                'hasnt', 'havent', 'dont', 'doesnt', 'didnt', 'cant', 'couldnt', 'shouldnt', 'mightnt', 'mustnt',
                'wouldnt']  # Список сокращений
lemmatizer = WordNetLemmatizer()  # Загрузка лемматизатора


# Функция опеределения части речи для лемматизации
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)


# Функция для очистки текста
def clean_text(text):
    if not isinstance(text, str):  # Проверяем, что текст является строкой
        return ''

    text = re.sub(r'<code>.*?</code>', '', text, flags=re.DOTALL)  # Удаляем блоки <code>
    text = re.sub(r'\`[^\`]*\`', '', text)  # Удаляем инлайн-код в Markdown
    text = re.sub(r'<[^>]*>', '', text)  # Удаляем оставшиеся HTML-теги
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)  # Удаляем Markdown-ссылки
    text = re.sub(r'"[^"]*"', '', text)  # Удаляем текст в кавычках
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы

    text = text.lower()  # Приводим текст к нижнему регистру
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Удаляем все кроме букв, цифр и пробелов

    words = text.split()  # Разбиваем текст на слова
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words if
             word not in stop_words and word not in contractions]  # Удаляем стоп-слова и сокращения, лемматизируем слова
    text = ' '.join(words)  # Объединяем слова обратно в текст

    return text

## Индексирование коллекции.

In [12]:
import math
from collections import defaultdict, Counter

In [30]:
inverted_index = defaultdict(dict)  # term -> {question_id: term_frequency}

question_lengths = {}  # question_id -> number_of_tokens

total_questions = len(questions_clear)  # Общее число вопросов

for _, row in questions_clear.iterrows():
    question_id = row['Id']
    text_body = str(row['Body'])

    # Разбиение на токены
    tokens = text_body.split()
    question_lengths[question_id] = len(tokens)

    # Подсчёт частот термов
    term_frequencies = Counter(tokens)

    # Заполнение инвертированного индекса
    for term, freq in term_frequencies.items():
        inverted_index[term][question_id] = freq

 IDF (Inverse Document Frequency) для BM25.

Это сглаженная обратная частотность, чтобы термы, встречающиеся во многих документах, давали меньший вклад:

$$IDF(t) = \log\left(\frac{N - df(t) + 0.5}{df(t) + 0.5} + 1\right)$$
где:

$N$ — общее число документов в корпусе

$df(t)$ — число документов, содержащих терм $t$ (document frequency)

In [33]:
# document_frequency_dict: term -> document_frequency
document_frequency_dict = {term: len(postings) for term, postings in inverted_index.items()}

average_question_length = sum(question_lengths.values()) / total_questions

# idf_bm25: term -> idf
idf_bm25 = {}
for term, df in document_frequency_dict.items():
    idf_bm25[term] = math.log((total_questions - df + 0.5) / (df + 0.5) + 1)

print(f"Всего вопросов: {total_questions}")
print(f"Уникальных токенов: {len(inverted_index)}")
print(f"Средняя длина вопроса: {average_question_length:.2f}\n")
print(f"10 токенов:")
for term in list(inverted_index.keys())[:10]:
    print(f"{term}: Частота: {len(inverted_index[term])}, IDF: {idf_bm25[term]:.2f}")

Всего вопросов: 1264216
Уникальных токенов: 1302678
Средняя длина вопроса: 44.27

10 токенов:
write: Частота: 111047, IDF: 2.43
database: Частота: 78291, IDF: 2.78
generation: Частота: 2201, IDF: 6.35
script: Частота: 60771, IDF: 3.04
sql: Частота: 37632, IDF: 3.51
want: Частота: 349727, IDF: 1.29
execute: Частота: 37967, IDF: 3.51
adobe: Частота: 1433, IDF: 6.78
air: Частота: 1146, IDF: 7.01
application: Частота: 115259, IDF: 2.40


## Реализация поиска

### Формула BM25

BM25 (Best Matching 25) — это функция оценки релевантности, которая используется в информационном поиске для ранжирования документов по их релевантности к запросу. Она основана на вероятностной модели и учитывает как частоту термов в документе, так и их распространённость в корпусе.

Для одного терма $t$ и документа $d$ формула BM25 выглядит следующим образом:

$$BM25(t, d) = IDF(t) \cdot \frac{f(t, d) \cdot (k_1 + 1)}{f(t, d) + k_1 \cdot (1 - b + b \cdot \frac{|d|}{\text{avgdl}})}$$

где:
* $IDF(t)$ — обратная частота документа терма $t$ (Inverse Document Frequency)
* $f(t, d)$ — частота терма $t$ в документе $d$
* $|d|$ — длина документа $d$ (число токенов)
* $\text{avgdl}$ — средняя длина документа в корпусе
* $k_1$ и $b$ — параметры, которые настраивают модель. Обычно $k_1$ берётся в диапазоне от 1.2 до 2.0, а $b$ — от 0.75 до 0.95.
* $k_1$ контролирует насыщение: при увеличении частоты терма в документе его вклад растёт, но с убывающей отдачей.
* $b$ контролирует влияние длины документа: чем длиннее документ, тем меньше вес терма.


Почему BM25 лучше «простого» TF–IDF.

Нормализация под длину документа
* Короткие документы не получают необоснованного преимущества только из-за малого объёма, а длинные — не «загружаются» частыми термами.

Сглаженный TF.
* Формула «успевает насыщаться»: при увеличении вклад растёт, но с убывающей отдачей, благодаря параметру.

Гибкость.
* Гиперпараметры позволяют подстроить модель под конкретный корпус: больше или меньше учитывать длину или переизбыток TF.

In [34]:
# Параметры BM25
k1 = 1.5
b = 0.75


# Функция BM25
def search_bm25(query, top_n=5):
    # Очистка и токенизация запроса
    query = clean_text(query)
    query_term_frequencies = Counter(query.split())

    results = defaultdict(float)  # question_id -> bm25_score

    # Подсчёт BM25 для каждого вопроса
    for term, qtf in query_term_frequencies.items():
        if term in inverted_index:
            idf = idf_bm25[term]
            postings = inverted_index[term]
            for question_id, tf in postings.items():
                dl = question_lengths[question_id]
                score = idf * (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (dl / average_question_length)))
                results[question_id] += score

    top_results = sorted(results.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return top_results

In [65]:
# Пример поиска
query = "<p>I've written a database generation script in <a href=""http://en.wikipedia.org/wiki/SQL"">SQL</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime"">Adobe AIR</a> application:</p>"
top_results = search_bm25(query, top_n=3)
for question_id, score in top_results:
    question_clear = questions_clear[questions_clear['Id'] == question_id]['Body'].values[0]
    question = questions[questions['Id'] == question_id]['Body'].values[0]
    print(f"ID: {question_id}, Score: {score:.4f}")
    print(f"Вопрос: {question_clear}")
    print(f"Оригинальный вопрос: {question}")
    print("-" * 80)

ID: 80, Score: 51.0315
Вопрос: write database generation script sql want execute adobe air application execute adobe air use follow method error generate however exists seem look first query semicolon remove query fails way call multiple query one statement
Оригинальный вопрос: <p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thumbnailID integer
    ,fileFormatID integer
    ,categoryID integer
    ,isFavorite boolean
    ,dateAdded date
    ,globalAccessCount integer
    ,lastAccessTime date
    ,downloadComplete boolean
    ,isNew boolean
    ,isSpotlight boolean
    ,duration varchar(30)
);
Create Tabl

In [66]:
# Пример поиска
query = "<p>I am porting the unity game in windows store game so have generated the windows store build from unity4.2.2 when i build the unity build solution from visual studio 2013 on windows 8.1 platform (retargetted the solution to 8.1 ) i am getting error at following line in AppManifest.xml</p>"
top_results = search_bm25(query, top_n=3)
for question_id, score in top_results:
    question_clear = questions_clear[questions_clear['Id'] == question_id]['Body'].values[0]
    question = questions[questions['Id'] == question_id]['Body'].values[0]
    print(f"ID: {question_id}, Score: {score:.4f}")
    print(f"Вопрос: {question_clear}")
    print(f"Оригинальный вопрос: {question}")
    print("-" * 80)

ID: 22473560, Score: 81.2029
Вопрос: port unity game window store game generate window store build unity422 build unity build solution visual studio 2013 window 81 platform retargetted solution 81 get error follow line appmanifestxml follow error warning get error 1 file content conform specify schema name attribute invalid value firgamea2 invalid accord datatype httpschemasmicrosoftcomappx2010manifeststpackagename pattern constraint fail ewindows game storefirshootxamlcsab ka shootbinarmdebugappxmanifestxml 10 13 sab ka shoot warn 2 name attribute invalid value firgamea2 invalid accord datatype httpschemasmicrosoftcomappx2010manifeststpackagename pattern constraint fail ewindows game storefirshootxamlcsab ka shootbinarmdebugappxmanifestxml 10 19 miscellaneous file warn 3 element package namespace httpschemasmicrosoftcomappx2010manifest invalid child element metadata namespace httpschemasmicrosoftcomdeveloperappx2012build ewindows game storefirshootxamlcsab ka shootbinarmdebugappxmanif

In [67]:
# Пример поиска
query = "I am building a test for an MVC5 controller method.  I'm using moq for the test.  What I'm interested in is how to test a controller method that requires authentication and uses the userid value not the username value to make decisions about what data to show to the browser/client."
top_results = search_bm25(query, top_n=3)
for question_id, score in top_results:
    question_clear = questions_clear[questions_clear['Id'] == question_id]['Body'].values[0]
    question = questions[questions['Id'] == question_id]['Body'].values[0]
    print(f"ID: {question_id}, Score: {score:.4f}")
    print(f"Вопрос: {question_clear}")
    print(f"Оригинальный вопрос: {question}")
    print("-" * 80)

ID: 28734300, Score: 62.0331
Вопрос: building test mvc5 controller method use moq test interested test controller method require authentication us userid value username value make decision data show browserclient research far considerable amount code available moq username much code userid value case look like guid use aspnet identity account management owin add well google facebook login use dependency injection use unity seem way accomplish test plus di enables use moq example test method look notnull return controller method getcurrentuserid method query useridentity object userid use code method supposedly get replace moq getcurrentuserid method virtual method helper class myhelper parameter constructor model object modelobject parameter constructor controller examplecontroller happens debugger visual studio 2013 test method object create createmodelobjectfromhelper method examplecontroller create line myhelper moqd object becomes null see object value statement statement myhelper 

## Извлечение ответа

In [104]:
# Функция для получения ответов на вопросы
def get_answers_id(question_id, top_n=5):
    answers_for_question = answers_clear[answers_clear['ParentId'] == question_id]
    best_answers_id = answers_for_question.nlargest(top_n, 'Score')['Id'].values
    return best_answers_id

In [114]:
# Найти вопросы с большим количеством ответов
questions_with_answers = answers_clear.groupby('ParentId').size().reset_index(name='AnswerCount')
questions_with_answers = questions_with_answers.sort_values(by='AnswerCount', ascending=False)
questions_with_answers

,ParentId,AnswerCount
5863,406760,408
421,38210,316
242,23930,129
1196,100420,100
441,40480,69
...,...,...
1102515,40140310,1
1102516,40140390,1
1102517,40140430,1
6,470,1


In [97]:
# Пример вопроса
question_id = 34048740
ans = answers_clear[answers_clear['ParentId'] == question_id]
ans

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
1734311,34048908,1159478.0,2015-12-02T17:15:15Z,34048740,391,big notation context use describe relationship...
1734315,34048956,6722.0,2015-12-02T17:17:30Z,34048740,88,bigo notation mean roughly give operation amou...
1734318,34049014,179910.0,2015-12-02T17:20:38Z,34048740,13,bigo analysis deal amount processing involve a...
1734365,34050296,2295834.0,2015-12-02T18:28:07Z,34048740,21,disagree servy slightly input program even obv...
1734421,34051419,1339987.0,2015-12-02T19:34:17Z,34048740,9,bigo relative seem intuit indeed write functio...
1734469,34052365,318964.0,2015-12-02T20:27:59Z,34048740,27,although bunch great answer let rephrase bit b...
1734542,34053948,5632236.0,2015-12-02T22:02:42Z,34048740,41,code time complexity proper cod current time p...
1734687,34056914,2728148.0,2015-12-03T02:39:48Z,34048740,6,one thing surprised mention yet bigo notation ...
1734864,34060817,4474419.0,2015-12-03T08:13:29Z,34048740,4,everyones correctly point define n answer reas...
1735885,34081440,2383529.0,2015-12-04T05:23:44Z,34048740,8,correctly described o1 constant time argue inp...


In [98]:
questions[questions['Id'] == question_id]['Body'].values[0]

'<p>Would this be classified as an O(1) algorithm for "Hello, World!" ??</p>\n\n<pre><code>public class Hello1\n{\n   public static void Main()\n   {\n      DateTime TwentyYearsLater = new DateTime(2035,01,01);\n      while ( DateTime.Now &lt; TwentyYearsLater )\n      { \n          System.Console.WriteLine("It\'s still not time to print the hello ...");\n      }\n      System.Console.WriteLine("Hello, World!");\n   }\n}\n</code></pre>\n\n<p>I\'m thinking of using the </p>\n\n<pre><code>DateTime TwentyYearsLater = new DateTime(2035,01,01);\nwhile ( DateTime.Now &lt; TwentyYearsLater )\n{ \n   // ... \n}\n</code></pre>\n\n<p>snippet of code as a busy loop to put in as a joke whenever someone asks for an algorithm of a certain complexity. Would this be correct?</p>\n'

In [101]:
# Пример поиска
query = r"""<p>Would this be classified as an O(1) algorithm for "Hello, World!" ??</p>"""
top_results = search_bm25(query, top_n=3)
for question_id, score in top_results:
    question_clear = questions_clear[questions_clear['Id'] == question_id]['Body'].values[0]
    question = questions[questions['Id'] == question_id]['Body'].values[0]
    print(f"ID: {question_id}, Score: {score:.4f}")
    print(f"Вопрос: {question_clear}")
    print(f"Оригинальный вопрос: {question}")
    print("-" * 80)

ID: 34048740, Score: 32.1568
Вопрос: would classify o1 algorithm think use snippet code busy loop put joke whenever someone asks algorithm certain complexity would correct
Оригинальный вопрос: <p>Would this be classified as an O(1) algorithm for "Hello, World!" ??</p>

<pre><code>public class Hello1
{
   public static void Main()
   {
      DateTime TwentyYearsLater = new DateTime(2035,01,01);
      while ( DateTime.Now &lt; TwentyYearsLater )
      { 
          System.Console.WriteLine("It's still not time to print the hello ...");
      }
      System.Console.WriteLine("Hello, World!");
   }
}
</code></pre>

<p>I'm thinking of using the </p>

<pre><code>DateTime TwentyYearsLater = new DateTime(2035,01,01);
while ( DateTime.Now &lt; TwentyYearsLater )
{ 
   // ... 
}
</code></pre>

<p>snippet of code as a busy loop to put in as a joke whenever someone asks for an algorithm of a certain complexity. Would this be correct?</p>

------------------------------------------------------------

In [112]:
# Получаем ответы на вопрос
answers_id = get_answers_id(top_results[0][0])
for answer_id in answers_id:
    answer_clear = answers_clear[answers_clear['Id'] == answer_id]['Body'].values[0]
    answer = answers[answers['Id'] == answer_id]['Body'].values[0]
    score = answers[answers['Id'] == answer_id]['Score'].values[0]
    print(f"ID: {answer_id}")
    print(f"Score: {score:.4f}")
    print(f"Ответ: {answer_clear}")
    print(f"Оригинальный ответ: {answer}")
    print("-" * 80)

ID: 34048908
Score: 391.0000
Ответ: big notation context use describe relationship size input function number operation need perform compute result input operation input output related use big notation nonsensical time operation take independent input operation isnone since relationship input number operation perform use big describe nonexistent relationship
Оригинальный ответ: <p>Big O notation in this context is being used to describe a relationship between the size of the input of a function and the number of operations that need to be performed to compute the result for that input.</p>

<p>Your operation has no input that the output can be related to, so using Big O notation is nonsensical.  The time that the operation takes is <em>independent</em> of the inputs of the operation (which is...none).  Since there <em>is</em> no relationship between the input and the number of operations performed, you can't use Big O to describe that non-existent relationship</p>

--------------------

In [128]:
# Функция Q&A
def QNA(query, top_questions=3, top_answers=3):
    top_results = search_bm25(query, top_n=top_questions)
    for question_id, score in top_results:
        print("*" * 36 + " Вопрос " + "*" * 36)
        question_clear = questions_clear[questions_clear['Id'] == question_id]['Body'].values[0]
        question = questions[questions['Id'] == question_id]['Body'].values[0]
        print(f"ID: {question_id}, Score: {score:.4f}")
        print(f"Вопрос: {question_clear}\n")
        print(f"Оригинальный вопрос: {question}")
        print("*" * 80)

        answers_id = get_answers_id(question_id, top_n=top_answers)
        for answer_id in answers_id:
            print("-" * 36 + " Ответ " + "-" * 36)
            answer_clear = answers_clear[answers_clear['Id'] == answer_id]['Body'].values[0]
            answer = answers[answers['Id'] == answer_id]['Body'].values[0]
            score = answers[answers['Id'] == answer_id]['Score'].values[0]
            print(f"ID: {answer_id}")
            print(f"Score: {score:.4f}")
            print(f"Ответ: {answer_clear}\n")
            print(f"Оригинальный ответ: {answer}")
            print("-" * 80)

        print('\n' + "=" * 80 + '\n')

In [124]:
questions[questions['Id'] == 12517000]['Body'].values[0]

'<p>I was really excited to see iOS 6 supports the Web Audio API, since we make HTML5 games.  However, I cannot get iOS 6 to play any sound at all using the Web Audio API with examples that work fine in desktop Chrome.</p>\n\n<p>Here is a HTML5 game with touch controls and playing audio via the Web Audio API (if present - if not it will fall back to HTML5 audio):</p>\n\n<p><a href="http://www.scirra.com/labs/sbios6b/">http://www.scirra.com/labs/sbios6b/</a></p>\n\n<p><em>Edit: @Srikumar suggested some workarounds.  I applied them at the version below.  It still does not work!</em></p>\n\n<p><a href="http://www.scirra.com/labs/sbios6f/">http://www.scirra.com/labs/sbios6f/</a></p>\n\n<p>Everything plays just fine on desktop Chrome, but iOS 6 emits no sound at all.  I\'m having trouble debugging it because I only do Windows development, and iOS 6 replaced the debug mode with remote web inspector, which apparently is not available on Safari for Windows.  Using a few alerts I did find it co

In [129]:
# Пример Q&A
query = r"""I cannot get iOS 6 to play any sound at all using the Web Audio API with examples that work fine in desktop Chrome."""
QNA(query, top_questions=3, top_answers=3)

************************************ Вопрос ************************************
ID: 12517000, Score: 63.7621
Вопрос: really excite see io 6 support web audio api since make html5 game however cannot get io 6 play sound use web audio api example work fine desktop chrome html5 game touch control play audio via web audio api present fall back html5 audio httpwwwscirracomlabssbios6b edit srikumar suggest workarounds apply version still work httpwwwscirracomlabssbios6f everything play fine desktop chrome io 6 emits sound trouble debug window development io 6 replace debug mode remote web inspector apparently available safari window use alert find correctly identifies web audio api us detects vorbis support fall back aac audio decodes buffer play error thrown hear nothing course try turn volume max codec problem io 6 play aac fine browse one m4as game play play fine visit direct safari look web audio api example io 6 httpchromiumgooglecodecomsvntrunksamplesaudiosampleshtml work others examp

## Тестирование было проведено на примерах, алгоритм работает хорошо, даже при достаточно малых запросах.

## 1. Введение

Цель лабораторной работы — реализовать базовый механизм поиска по датасету с помощью классических методов информационного поиска (ключевые слова, TF-IDF, BM25). Оценить качество поиска с помощью простых метрик.

По результатам анализа задачи, было решено использовать не соединённые датасеты, так как это только усложнит задачу. Вопросы и ответы были очищены в предыдущей лабораторной работе, поэтому в данной работе мы будем использовать очищенные датасеты Questions_clear.csv и Answers_clear.csv.

## 2. Индексирование коллекции
Во время работы была использована сторонняя библиотека collections. defaultdict необходима, так как она позволяет создавать словари с заданным значением по умолчанию. А Counter позволяет подсчитывать количество вхождений каждого терма в документе.

Были реализованы следующие шаги:
2.1 Инвертированный индекс -  позволяет быстро находить документы, содержащие определённые термы. Каждому терму сопоставляется список документов, в которых он встречается, а также частота его появления в каждом документе.

2.2 TF и IDF - позволяет оценить важность терма в документе относительно всего корпуса. TF (Term Frequency) — частота терма в документе, IDF (Inverse Document Frequency) — обратная частота терма в корпусе. IDF вычислялся по формуле для BM25. Данная формула сглаживает IDF и делает его менее чувствительным к термам, которые встречаются во всех документах, в отличие от классической TF-IDF.

## 3. Реализация поиска
Поиск реализован на основе алгоритма BM25, учитывающего частоту термов и длину документа. Алгоритм возвращает наиболее релевантные вопросы по входному запросу:

* Запрос очищается и лемматизируется так же, как и документы;

* Строится вектор запроса;

* Для каждого документа вычисляется BM25-оценка;

* Документы сортируются по убыванию релевантности;

* Выдаются top-N наиболее подходящих вопросов.

BM25 используется со стандартными гиперпараметрами.

## 4. Извлечение ответа
После получения релевантных вопросов из датасета Questions_clear.csv:

* По ID каждого вопроса извлекаются все соответствующие ответы из Answers_clear.csv;

* Из них выбираются ответы с максимальным рейтингом.

Это позволяет по запросу пользователя возвращать не только похожие вопросы, но и полноценные Q&A пары.

## 5. Оценка качества
Оценка качества поиска проводилась вручную. Были составлены тестовые запросы, которые проверялись на релевантность. Проблемы возникали лишь с маленькими запросами, но в целом алгоритм работал хорошо. Тяжело оценить качество на достаточном малом датасете.

## 6. Заключение
Реализованный поиск по корпусу вопросов Stack Overflow с помощью BM25 показал удовлетворительное качество на небольшом тестовом наборе.
